## Постановка задачи
Разберем архитектуру MobileNet и проведем частичное обучение, экспорт и импорт модели, затем дообучение.

Перейдем от задачи классификации изображения к задаче локализации объекта на изображении при помощи якорей.

Построим предсказание по обученной нейросети и проведем оценку качества предсказания по коэффициенту сходства.

Данные:
* https://video.ittensive.com/machine-learning/clouds/train.csv.gz (54 Мб)
* https://video.ittensive.com/machine-learning/clouds/train_images_small.tar.gz (212 Мб)

Соревнование: https://www.kaggle.com/c/understanding_cloud_organization/

![](mobilenet.01.png)
![](mobilenet.02.jpeg)
![](anchors.png)

### Подключение библиотек

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Activation
from keras.layers import Conv2D, BatchNormalization, Dropout
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.regularizers import l2
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

Using TensorFlow backend.


### Используемые функции
Разобьем изображение на 5x5 квадратов и будем предсказывать наличие облаков заданной формы в каждом из квадратов.

При вычислении коэффициента Дайса "растянем" предсказанную маску до размеров изображения.

In [2]:
batch_size = 10
filesDir = "train_images_small"
image_x = 525 # 525
image_y = 350 # 350
image_ch = 3 # 3
mask_x = 5
mask_y = 5
def mask_rate (a, x, y):
    b = a//1400 + 0.0
    return round(x*(b*x//2100) + y*(a%1400)//1400).astype("uint32")

def calc_mask (px, x=image_x, y=image_y):
    p = np.array([int(n) for n in px.split(' ')]).reshape(-1,2)
    mask = np.zeros(x*y, dtype='uint8')
    for i, l in p:
        mask[mask_rate(i, x, y):mask_rate(l+i, x, y)+1] = 1
    return mask.reshape(y,x).transpose()

def calc_dice (x):
    dice = 0
    px = x["EncodedPixels"] 
    if px != px and x["target"] == 0:
        dice = 1
    elif px == px and x["target"] == 1:
        mask = calc_mask(px).flatten()
        target = np.kron(np.array(x["MaskPixels"].split(" ")).reshape(mask_x,
            mask_y).astype("i1"),
            np.ones((image_y//mask_y, image_x//mask_x),
            dtype="i1")).transpose().flatten()
        dice = 2*np.sum(target[mask==1])/(np.sum(target)+np.sum(mask))
    return dice

def load_y (df):
    y = [[0]]*len(df)
    for i, ep in enumerate(df["EncodedPixels"]):
        if ep == ep:
            y[i] = calc_mask(ep, mask_x, mask_y).transpose().flatten()
        else:
            y[i] = np.zeros(mask_x*mask_y, dtype="i1")
    return np.array(y).reshape(len(df), mask_y, mask_x, 1)

def load_x (df):
    x = [[]]*len(df)
    for j, file in enumerate(df["Image"]):
        img = image.load_img(os.path.join(filesDir, file),
                     target_size=(image_y, image_x))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        x[j] = preprocess_input(img)
    return np.array(x).reshape(len(df), image_y, image_x, image_ch)

def load_data (df, batch_size):
    while True:
        batch_start = 0
        batch_end = batch_size
        while batch_start < len(df):
            limit = min(batch_end, len(df))
            yield (load_x(df[batch_start:limit]),
                   load_y(df[batch_start:limit]))
            batch_start += batch_size   
            batch_end += batch_size

def draw_prediction (prediction):
    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_subplot(1,1,1)
    ax.hist(prediction[0])
    ax.set_title("Fish")
    plt.show()

### Загрузка данных

In [3]:
data = pd.read_csv('https://video.ittensive.com/machine-learning/clouds/train.csv.gz')

In [4]:
data["Image"] = data["Image_Label"].str.split("_").str[0]
data["Label"] = data["Image_Label"].str.split("_").str[1]
data.drop(labels=["Image_Label"], axis=1, inplace=True)
data_fish = data[data["Label"] == "Fish"]
print (data_fish.head())

                                        EncodedPixels        Image Label
0   264918 937 266318 937 267718 937 269118 937 27...  0011165.jpg  Fish
4   233813 878 235213 878 236613 878 238010 881 23...  002be4f.jpg  Fish
8   3510 690 4910 690 6310 690 7710 690 9110 690 1...  0031ae9.jpg  Fish
12                                                NaN  0035239.jpg  Fish
16  2367966 18 2367985 2 2367993 8 2368002 62 2369...  003994e.jpg  Fish


### Разделение данных
Разделим всю выборку на 2 части случайным образом: 80% - для обучения модели, 20% - для проверки точности модели.

In [5]:
train, test = train_test_split(data_fish, test_size=0.2)
train = pd.DataFrame(train)
test = pd.DataFrame(test)
del data
print (train.head())

                                           EncodedPixels        Image Label
336                                                  NaN  0493d31.jpg  Fish
19640                                                NaN  e2e4fa9.jpg  Fish
11728                                                NaN  85e5f09.jpg  Fish
21896  744166 585 745566 585 746966 585 748366 585 74...  fcb25d2.jpg  Fish
13636  376601 279 378001 279 379401 279 380801 279 38...  9b25c0d.jpg  Fish


### MobileNetV2
Используем обученную нейросеть и bn/dropout/softmax-слой поверх. Обучим модель из последнего слоя 50 эпох, сохраним обученные данные, затем загрузим их и продолжим обучение.

Используем раннюю остановку и сохранение модели после каждой эпохи обучения.

In [6]:
base_model = MobileNetV2(weights='imagenet', include_top=False,
                        input_shape=(image_y, image_x, image_ch))
base_model_output = base_model.predict_generator(load_data(train, 1),
                        steps=len(train), verbose=1)
top_model = Sequential([
    Flatten(input_shape=base_model_output.shape[1:]),
    BatchNormalization(),
    Dropout(0.5),
    Activation("softmax"),
    Dense(mask_x * mask_y)
])
top_model.compile(optimizer=optimizers.Nadam(lr=0.05),
                 loss="mean_absolute_error")

c:\users\nikolay\appdata\local\programs\python\python37\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
W0329 14:40:28.480491  8572 deprecation_wrapper.py:119] From c:\users\nikolay\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



4436/4436 [==============================] - 5813s 1s/step


W0329 16:21:53.125513  8572 deprecation.py:506] From c:\users\nikolay\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
top_model.fit(base_model_output,
             load_y(train).reshape(len(train), -1), epochs=100,
             callbacks=[EarlyStopping(monitor="loss",
                min_delta=0.0001, patience=5, verbose=1, mode="auto"),
                ModelCheckpoint("mobilenet.clouds.h5", mode="auto",
                monitor="val_loss", verbose=1)])

Epoch 1/100
4436/4436 [==============================] - 139s 31ms/step - loss: 0.3664

Epoch 00001: saving model to mobilenet.clouds.h5
Epoch 2/100
4436/4436 [==============================] - 112s 25ms/step - loss: 0.3430

Epoch 00002: saving model to mobilenet.clouds.h5
Epoch 3/100
4436/4436 [==============================] - 110s 25ms/step - loss: 0.3309

Epoch 00003: saving model to mobilenet.clouds.h5
Epoch 4/100
4436/4436 [==============================] - 106s 24ms/step - loss: 0.3239

Epoch 00004: saving model to mobilenet.clouds.h5
Epoch 5/100
4436/4436 [==============================] - 108s 24ms/step - loss: 0.3199

Epoch 00005: saving model to mobilenet.clouds.h5
Epoch 6/100
4436/4436 [==============================] - 112s 25ms/step - loss: 0.3152

Epoch 00006: saving model to mobilenet.clouds.h5
Epoch 7/100
4436/4436 [==============================] - 124s 28ms/step - loss: 0.3146

Epoch 00007: saving model to mobilenet.clouds.h5
Epoch 8/100
4436/4436 [==================

### Продолжение обучения
Загрузим модель из файла (структура + веса) и продолжим обучение

In [8]:
del top_model
top_model = keras.models.load_model("mobilenet.clouds.h5")

In [9]:
top_model.fit(base_model_output,
             load_y(train).reshape(len(train), -1), epochs=100,
             callbacks=[EarlyStopping(monitor="loss",
                min_delta=0.0001, patience=5, verbose=1, mode="auto"),
                ModelCheckpoint("mobilenet.clouds.h5", mode="auto",
                monitor="val_loss", verbose=1)])

Epoch 1/100
4436/4436 [==============================] - 140s 32ms/step - loss: 0.3022

Epoch 00001: saving model to mobilenet.clouds.h5
Epoch 2/100
4436/4436 [==============================] - 130s 29ms/step - loss: 0.2996

Epoch 00002: saving model to mobilenet.clouds.h5
Epoch 3/100
4436/4436 [==============================] - 134s 30ms/step - loss: 0.2980

Epoch 00003: saving model to mobilenet.clouds.h5
Epoch 4/100
4436/4436 [==============================] - 133s 30ms/step - loss: 0.2994

Epoch 00004: saving model to mobilenet.clouds.h5
Epoch 5/100
4436/4436 [==============================] - 132s 30ms/step - loss: 0.2939

Epoch 00005: saving model to mobilenet.clouds.h5
Epoch 6/100
4436/4436 [==============================] - 134s 30ms/step - loss: 0.2976

Epoch 00006: saving model to mobilenet.clouds.h5
Epoch 7/100
4436/4436 [==============================] - 134s 30ms/step - loss: 0.2988

Epoch 00007: saving model to mobilenet.clouds.h5
Epoch 8/100
4436/4436 [==================

Соберем модель из обученного финального слоя и базовой модели

In [10]:
model = Model(inputs=base_model.input,
             outputs=top_model(base_model.output))
model.compile(optimizer="adam", loss="mean_absolute_error")
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 350, 525, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 351, 527, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 175, 263, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 175, 263, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

### Построение предсказания

In [11]:
prediction = model.predict_generator(load_data(test, 1),
                                    steps=len(test), verbose=1)

1110/1110 [==============================] - 1462s 1s/step


In [12]:
prediction = (prediction>0.5).astype("i1")

In [13]:
print (prediction[0:10])
print (load_y(test.head(10)).reshape(10, -1))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[0 1 1 1 1 0 1 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 0 1 1]
 [1 1 1 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [20]:
target = []
masks = []
for i,vals in enumerate(prediction):
    if vals.sum() > 4:
        targ = 1
    else:
        targ = 0
    target.append(targ)
    masks.append(np.array2string(vals.flatten().astype("int8"),
                                separator=" ")[1:-1])
test["MaskPixels"] = masks
test["target"] = target
print (test[test["target"]>0][["EncodedPixels","MaskPixels"]].head(20))

                                           EncodedPixels  \
4324   40928 756 42328 756 43728 756 45128 756 46528 ...   
2216   1 428 430 2 433 2 437 5 444 1 450 3 455 1 1401...   
4656                                                 NaN   
4660   67850 467 69250 467 70650 467 72050 467 73450 ...   
5632                                                 NaN   
9096                                                 NaN   
7876                                                 NaN   
7240   1167092 388 1168492 388 1169892 388 1171292 38...   
552    169 809 1569 721 2291 87 2969 809 4369 809 576...   
6800   748348 650 749748 650 751148 650 752548 650 75...   
20688  503056 292 504456 292 505856 292 507256 292 50...   
2208   3000 367 4400 367 5800 367 7200 367 8600 367 1...   
16520  54644 463 56044 463 57444 463 58844 463 60244 ...   
7512   3104 355 4504 355 5904 355 7304 355 8704 355 1...   
308                                                  NaN   
5376                                    

### Расчет точности предсказания
Нет облаков - 0.5, MLP - 0.3, CONV/VGG - 0.48, AlexNet - 0.21, Inception - 0.4, ResNet - 0.52, VGG16+Inception+ResNet - 0.54

In [17]:
dice = test.apply(calc_dice, axis=1, result_type="expand")
print ("Keras, MobileNet:", round(dice.mean(), 3))

Keras, MobileNet: 0.472
